In [17]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [18]:
msn = pd.read_csv(r'Data_MSN.csv',index_col=0)
msn.columns = ['VN_Index','MSN_Price','Risk_free']
msn.dropna(inplace=True,axis=0)

In [20]:
def fix_format_number(df, columns):
    list_fixed = []
    for i in df[columns]:
        i = i.replace(',','.')
        list_fixed.append(float(i))

    return list_fixed

In [21]:
msn['VN_Index'] = fix_format_number(msn, 'VN_Index')
msn['MSN_Price'] = fix_format_number(msn, 'MSN_Price')
msn['Risk_free'] = fix_format_number(msn, 'Risk_free')

In [23]:
df_msn = pd.DataFrame({},index=msn.index,columns=msn.columns)
j = -1
for i in range((len(msn)-1),-1,-1):
    j+= 1
    df_msn.iloc[j] = msn.iloc[i]  

In [24]:
date = list(df_msn.index)
date.reverse()
df_msn.index = date

In [25]:
df_msn

,VN_Index,MSN_Price,Risk_free
4/1/2016,574.41,76950.0,0.023
5/1/2016,569.94,76850.0,0.023
6/1/2016,574.57,76750.0,0.023
7/1/2016,565.36,76000.0,0.023
8/1/2016,560.05,77500.0,0.023
...,...,...,...
25/12/2020,1084.42,82500.0,0.023
28/12/2020,1091.33,82400.0,0.023
29/12/2020,1099.49,83200.0,0.023
30/12/2020,1097.54,83500.0,0.023


In [26]:
df_msn['Risk_Market'] = df_msn.VN_Index.pct_change()
df_msn['Risk_require'] = df_msn.MSN_Price.pct_change()
df_msn.dropna(axis=0,inplace=True)

In [27]:
df_msn

,VN_Index,MSN_Price,Risk_free,Risk_Market,Risk_require
5/1/2016,569.94,76850.0,0.023,-0.007782,-0.001300
6/1/2016,574.57,76750.0,0.023,0.008124,-0.001301
7/1/2016,565.36,76000.0,0.023,-0.016029,-0.009772
8/1/2016,560.05,77500.0,0.023,-0.009392,0.019737
11/1/2016,557.87,76500.0,0.023,-0.003893,-0.012903
...,...,...,...,...,...
25/12/2020,1084.42,82500.0,0.023,0.015831,-0.002418
28/12/2020,1091.33,82400.0,0.023,0.006372,-0.001212
29/12/2020,1099.49,83200.0,0.023,0.007477,0.009709
30/12/2020,1097.54,83500.0,0.023,-0.001774,0.003606


In [28]:
df_msn['Risk_premium_MSN'] = df_msn['Risk_require'] - df_msn['Risk_free'] 
df_msn['Market_risk_premium'] = df_msn['Risk_Market'] - df_msn['Risk_free'] 

In [29]:
risk_msn = df_msn['Risk_premium_MSN']
premium_risk = df_msn['Market_risk_premium']

premium_risk, risk_msn = np.array(premium_risk, dtype=float), np.array(risk_msn, dtype=float)
premium_risk = sm.add_constant(premium_risk)
# Mô hình OLS
results = sm.OLS(risk_msn,premium_risk).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     118.1
Date:                Tue, 21 Dec 2021   Prob (F-statistic):           2.40e-26
Time:                        16:01:12   Log-Likelihood:                 3060.8
No. Observations:                1251   AIC:                            -6118.
Df Residuals:                    1249   BIC:                            -6107.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0091      0.001     -6.548      0.0

In [30]:
df_msn.to_excel('MSN_CAPM.xlsx')